
---

# **Chapter 29: Widget Testing**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Find widgets using various finder strategies (text, type, key, icon)
- Simulate user interactions including tap, scroll, drag, and text input
- Test widget rendering across different screen sizes and orientations
- Implement golden tests for visual regression detection
- Test animations and transitions effectively
- Handle complex widget trees and deep nesting
- Test platform-specific widgets (Material vs. Cupertino)
- Implement widget tests for custom painters and render objects

---

## **Prerequisites**

- Completed Chapter 27: Testing Fundamentals (test structure, AAA pattern)
- Completed Chapter 28: Unit Testing (async testing, mocking)
- Understanding of Flutter widget lifecycle and BuildContext
- Familiarity with Material Design and Cupertino widgets
- `flutter_test` package available in dev_dependencies

---

## **29.1 Widget Testing Fundamentals**

Widget tests verify that widgets render correctly and respond to user interactions. They run in a headless environment faster than integration tests but slower than unit tests.

### **Basic Widget Test Structure**

```dart
// File: test/presentation/widgets/counter_widget_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';
import 'package:my_app/presentation/widgets/counter_widget.dart';

void main() {
  // testWidgets is the entry point for widget tests
  // WidgetTester provides utilities to interact with the widget tree
  testWidgets('CounterWidget increments when button tapped', 
      (WidgetTester tester) async {
    
    // ARRANGE: Build the widget tree
    // pumpWidget renders the widget and adds it to the test environment
    await tester.pumpWidget(
      MaterialApp(
        home: Scaffold(
          body: CounterWidget(),
        ),
      ),
    );
    
    // MaterialApp provides the Material context needed for Material widgets
    // Scaffold provides the basic app structure
    // CounterWidget is the widget under test
    
    // ACT: Find the button and tap it
    // find.text searches for a Text widget with the given string
    final buttonFinder = find.text('Increment');
    
    // Verify the button exists before tapping
    expect(buttonFinder, findsOneWidget);
    
    // tester.tap simulates a user tap on the found widget
    await tester.tap(buttonFinder);
    
    // pump() triggers a frame rebuild after state change
    // Widgets don't rebuild automatically in tests after setState
    await tester.pump();
    
    // ASSERT: Verify the counter incremented
    // find.text('1') looks for Text widget displaying "1"
    expect(find.text('1'), findsOneWidget);
    
    // Verify the old value is gone
    expect(find.text('0'), findsNothing);
  });
}

// Implementation being tested
class CounterWidget extends StatefulWidget {
  @override
  _CounterWidgetState createState() => _CounterWidgetState();
}

class _CounterWidgetState extends State<CounterWidget> {
  int _counter = 0;
  
  @override
  Widget build(BuildContext context) {
    return Column(
      children: [
        Text('$_counter'), // Displays current count
        ElevatedButton(
          onPressed: () {
            setState(() {
              _counter++; // Triggers rebuild
            });
          },
          child: Text('Increment'),
        ),
      ],
    );
  }
}
```

**Explanation:**

- **`testWidgets()`**: Special function for widget tests. Provides a `WidgetTester` that can build, find, and interact with widgets. The test runs in a fake asynchronous zone.
- **`pumpWidget()`**: Renders the widget tree. You must wrap widgets in `MaterialApp` or `CupertinoApp` if they depend on theme, navigation, or platform-specific widgets.
- **`find` object**: Collection of finder functions that search the widget tree. Returns a `Finder` that can be used with `expect()` or interaction methods.
- **`tester.tap()`**: Simulates a user tap. Requires `await` because it's asynchronous. After state-changing interactions, you must call `pump()` to rebuild.
- **`pump()`**: Triggers a single frame rebuild. After `setState()`, `pump()` updates the tree so you can assert on the new state.
- **`findsOneWidget`**: Matcher ensuring exactly one widget matches. Prevents accidental matches of multiple similar widgets.
- **`findsNothing`**: Matcher ensuring no widgets match, verifying the old state disappeared.

### **Understanding the Test Environment**

```dart
// File: test/environment/widget_test_environment.dart

import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Widget Test Environment', () {
    testWidgets('understanding the test surface', (WidgetTester tester) async {
      // The test environment provides a simulated 800x600 screen by default
      // You can verify this:
      final size = tester.binding.window.physicalSize;
      print('Screen size: $size'); // Size(800.0, 600.0) by default
      
      // Build a widget that uses MediaQuery
      await tester.pumpWidget(
        MaterialApp(
          home: Builder(
            builder: (context) {
              final mq = MediaQuery.of(context);
              return Text(
                'Size: ${mq.size.width}x${mq.size.height}',
              );
            },
          ),
        ),
      );
      
      // Default test size is 800x600 logical pixels (not physical)
      expect(find.text('Size: 800.0x600.0'), findsOneWidget);
    });
    
    testWidgets('custom screen sizes', (WidgetTester tester) async {
      // Set a specific screen size for responsive testing
      tester.binding.window.physicalSizeTestValue = Size(1080, 1920);
      // This simulates a phone screen in portrait
      
      // Also set device pixel ratio for realistic density
      tester.binding.window.devicePixelRatioTestValue = 2.0;
      
      await tester.pumpWidget(
        MaterialApp(
          home: ResponsiveWidget(),
        ),
      );
      
      // Test responsive behavior at this size
      // ...
      
      // IMPORTANT: Reset after test to avoid affecting other tests
      addTearDown(() {
        tester.binding.window.clearPhysicalSizeTestValue();
        tester.binding.window.clearDevicePixelRatioTestValue();
      });
    });
    
    testWidgets('platform simulation', (WidgetTester tester) async {
      // Test different platforms
      debugDefaultTargetPlatformOverride = TargetPlatform.iOS;
      
      await tester.pumpWidget(
        MaterialApp(
          home: PlatformSpecificWidget(),
        ),
      );
      
      // Verify iOS-specific behavior
      
      // Reset platform
      addTearDown(() {
        debugDefaultTargetPlatformOverride = null;
      });
    });
  });
}

class ResponsiveWidget extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    final width = MediaQuery.of(context).size.width;
    // Return different layouts based on width
    return width > 600 ? Text('Tablet') : Text('Phone');
  }
}

class PlatformSpecificWidget extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    // Uses platform-specific widgets internally
    return Switch.adaptive(value: true, onChanged: (_) {});
  }
}
```

**Explanation:**

- **Default size**: Widget tests run in an 800x600 logical pixel environment by default. This is larger than most phones, so always test at realistic device sizes for responsive layouts.
- **`physicalSizeTestValue`**: Overrides the screen size for testing responsive breakpoints. Essential for testing tablet vs. phone layouts.
- **`devicePixelRatioTestValue`**: Simulates different screen densities (1.0 for standard, 2.0 for retina, 3.0 for high density).
- **`addTearDown()`**: Registers cleanup code that runs after the test. Always reset window overrides to prevent test pollution.
- **`debugDefaultTargetPlatformOverride`**: Simulates iOS or Android platform-specific behavior. Affects `Switch.adaptive`, platform-specific scroll physics, and default themes.

---

## **29.2 Finder Strategies**

Finding the correct widget is crucial for interaction. Flutter provides multiple finder types for different scenarios.

### **Common Finder Methods**

```dart
// File: test/presentation/widgets/finder_examples_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Finder Strategies', () {
    testWidgets('find by text content', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Column(
              children: [
                Text('Hello World'),
                Text('Hello', style: TextStyle(fontSize: 24)),
                Text('World'),
              ],
            ),
          ),
        ),
      );
      
      // find.text matches exact string
      expect(find.text('Hello World'), findsOneWidget);
      
      // find.textContaining matches substring
      expect(find.textContaining('Hello'), findsNWidgets(2));
      // findsNWidgets expects exactly N matches
      
      // find.widgetWithText finds parent widget containing text
      expect(find.widgetWithText(Text, 'Hello World'), findsOneWidget);
    });
    
    testWidgets('find by type', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Column(
              children: [
                ElevatedButton(onPressed: () {}, child: Text('Button 1')),
                ElevatedButton(onPressed: () {}, child: Text('Button 2')),
                TextButton(onPressed: () {}, child: Text('Button 3')),
                Icon(Icons.star),
              ],
            ),
          ),
        ),
      );
      
      // find.byType matches widget class exactly
      expect(find.byType(ElevatedButton), findsNWidgets(2));
      expect(find.byType(TextButton), findsOneWidget);
      
      // find.byIcon matches Icon widgets by icon data
      expect(find.byIcon(Icons.star), findsOneWidget);
      
      // find.byWidgetPredicate for custom logic
      expect(
        find.byWidgetPredicate(
          (widget) => widget is ElevatedButton && 
                     (widget.child as Text).data?.startsWith('Button') == true,
        ),
        findsNWidgets(2),
      );
    });
    
    testWidgets('find by key', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Column(
              children: [
                Text('First', key: Key('first_text')),
                Text('Second', key: ValueKey('second_text')),
                // ValueKey is preferred over Key for type safety
                Container(key: Key('container')),
              ],
            ),
          ),
        ),
      );
      
      // find.byKey matches Key or ValueKey
      expect(find.byKey(Key('first_text')), findsOneWidget);
      expect(find.byKey(ValueKey('second_text')), findsOneWidget);
      
      // Keys are the most reliable way to find widgets
      // especially when text or structure might change
    });
    
    testWidgets('find by semantics', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Column(
              children: [
                TextField(
                  decoration: InputDecoration(labelText: 'Username'),
                ),
                ElevatedButton(
                  onPressed: () {},
                  child: Text('Submit'),
                ),
              ],
            ),
          ),
        ),
      );
      
      // find.bySemanticsLabel finds widgets by accessibility label
      expect(find.bySemanticsLabel('Username'), findsOneWidget);
      
      // find.ancestor finds widgets that contain other widgets
      expect(
        find.ancestor(
          of: find.text('Submit'),
          matching: find.byType(ElevatedButton),
        ),
        findsOneWidget,
      );
      
      // find.descendant finds widgets inside other widgets
      expect(
        find.descendant(
          of: find.byType(Scaffold),
          matching: find.byType(Column),
        ),
        findsOneWidget,
      );
    });
    
    testWidgets('chained finders', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: ListView.builder(
            itemCount: 10,
            itemBuilder: (context, index) {
              return ListTile(
                title: Text('Item $index'),
                trailing: IconButton(
                  icon: Icon(Icons.delete),
                  onPressed: () {},
                ),
              );
            },
          ),
        ),
      );
      
      // Find specific item's delete button
      final item5Finder = find.text('Item 5');
      final deleteButton = find.descendant(
        of: item5Finder,
        matching: find.byIcon(Icons.delete),
      );
      
      expect(deleteButton, findsOneWidget);
      
      // Alternative: Find by widget with specific parent
      final specificDelete = find.widgetWithIcon(ListTile, Icons.delete);
      // But this finds all ListTiles with delete icons
      
      // Use first/last/at getters for multiple matches
      expect(deleteButton.first, findsOneWidget);
    });
    
    testWidgets('evaluating finders', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Text('Test'),
        ),
      );
      
      // Get the actual widget from finder
      final textFinder = find.text('Test');
      final Text textWidget = tester.widget<Text>(textFinder);
      expect(textWidget.data, equals('Test'));
      
      // Get list of all matching widgets
      final widgets = tester.widgetList(find.byType(Text));
      expect(widgets.length, equals(1));
      
      // Get center location of widget (for gestures)
      final center = tester.getCenter(find.text('Test'));
      expect(center, isA<Offset>());
    });
  });
}
```

**Explanation:**

- **`find.text()`**: Most common finder. Matches exact text content. Be careful with dynamic text (dates, numbers) that might change.
- **`find.byType()`**: Matches widget class. Use with specific types (e.g., `ElevatedButton` not just `Button`). Returns all instances.
- **`find.byKey()`**: Most reliable for stable tests. Use `ValueKey<String>` for type safety. Essential when text is dynamic or internationalized.
- **`find.byIcon()`**: Matches `Icon` widgets by `IconData`. Useful for icon buttons.
- **`find.ancestor()` / `find.descendant()`**: Navigate the widget tree vertically. Find parents or children of specific widgets.
- **`find.widgetWithText()`**: Finds a widget type that contains specific text. Useful for finding `ListTile` by its title.
- **`tester.widget()`**: Evaluates a finder to get the actual widget instance. Allows inspecting properties like `style`, `data`, or custom fields.
- **`tester.widgetList()`**: Returns all matching widgets as an iterable for bulk assertions.

---

## **29.3 Simulating User Interactions**

Widget tests simulate user gestures to verify interactive behavior.

### **Tap, Long Press, and Drag**

```dart
// File: test/presentation/widgets/interactions_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('User Interactions', () {
    testWidgets('button tap triggers callback', (WidgetTester tester) async {
      var tapped = false;
      
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: ElevatedButton(
              onPressed: () => tapped = true,
              child: Text('Tap me'),
            ),
          ),
        ),
      );
      
      // Verify initial state
      expect(tapped, isFalse);
      
      // Tap the button
      await tester.tap(find.text('Tap me'));
      await tester.pump(); // Rebuild after state change
      
      expect(tapped, isTrue);
    });
    
    testWidgets('long press triggers separate callback', 
        (WidgetTester tester) async {
      var normalTapped = false;
      var longPressed = false;
      
      await tester.pumpWidget(
        MaterialApp(
          home: GestureDetector(
            onTap: () => normalTapped = true,
            onLongPress: () => longPressed = true,
            child: Container(
              width: 100,
              height: 100,
              color: Colors.blue,
            ),
          ),
        ),
      );
      
      // Long press at center of container
      await tester.longPress(find.byType(Container));
      await tester.pump();
      
      expect(normalTapped, isFalse);
      expect(longPressed, isTrue);
    });
    
    testWidgets('double tap detection', (WidgetTester tester) async {
      var doubleTapped = false;
      
      await tester.pumpWidget(
        MaterialApp(
          home: GestureDetector(
            onDoubleTap: () => doubleTapped = true,
            child: Text('Double tap me'),
          ),
        ),
      );
      
      // tap twice quickly
      await tester.tap(find.text('Double tap me'));
      await tester.tap(find.text('Double tap me'));
      await tester.pump();
      
      expect(doubleTapped, isTrue);
    });
    
    testWidgets('drag gestures', (WidgetTester tester) async {
      var dragEndDetails = '';
      
      await tester.pumpWidget(
        MaterialApp(
          home: GestureDetector(
            onHorizontalDragEnd: (details) {
              dragEndDetails = 
                  'velocity: ${details.primaryVelocity}';
            },
            child: Container(
              width: 300,
              height: 300,
              color: Colors.green,
            ),
          ),
        ),
      );
      
      // Drag from left to right
      await tester.drag(
        find.byType(Container),
        Offset(100, 0), // Drag 100 pixels right, 0 vertical
      );
      await tester.pump();
      
      expect(dragEndDetails, contains('velocity'));
    });
    
    testWidgets('scroll interaction', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: ListView.builder(
            itemCount: 100,
            itemBuilder: (context, index) => ListTile(
              title: Text('Item $index'),
            ),
          ),
        ),
      );
      
      // Verify first item visible
      expect(find.text('Item 0'), findsOneWidget);
      
      // Scroll down by 300 pixels
      await tester.scroll(
        find.byType(ListView),
        const Offset(0, -300), // Negative Y scrolls down
      );
      await tester.pump();
      
      // Item 0 should be scrolled off screen
      expect(find.text('Item 0'), findsNothing);
      expect(find.text('Item 5'), findsOneWidget); // Approximate
    });
    
    testWidgets('fling (swipe) with velocity', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: PageView(
            children: [
              Container(color: Colors.red, child: Text('Page 1')),
              Container(color: Colors.blue, child: Text('Page 2')),
            ],
          ),
        ),
      );
      
      // Verify first page
      expect(find.text('Page 1'), findsOneWidget);
      expect(find.text('Page 2'), findsNothing);
      
      // Fling left to go to next page
      await tester.fling(
        find.byType(PageView),
        Offset(-300, 0), // Negative X flings left
        800, // Velocity in pixels per second
      );
      
      // Wait for animation to complete
      await tester.pumpAndSettle();
      // pumpAndSettle waits for all animations to finish
      
      expect(find.text('Page 2'), findsOneWidget);
    });
    
    testWidgets('pinch gesture simulation', (WidgetTester tester) async {
      var scale = 1.0;
      
      await tester.pumpWidget(
        MaterialApp(
          home: GestureDetector(
            onScaleUpdate: (details) {
              scale = details.scale;
            },
            child: Transform.scale(
              scale: scale,
              child: Text('Pinch me'),
            ),
          ),
        ),
      );
      
      // Simulate two-finger pinch using startGesture
      final center = tester.getCenter(find.text('Pinch me'));
      
      // Create two separate touch pointers
      final pointer1 = await tester.startGesture(
        center.translate(-50, 0),
        pointer: 1,
      );
      final pointer2 = await tester.startGesture(
        center.translate(50, 0),
        pointer: 2,
      );
      
      // Move fingers apart (zoom in)
      await pointer1.moveBy(Offset(-50, 0));
      await pointer2.moveBy(Offset(50, 0));
      await tester.pump();
      
      // Cleanup
      await pointer1.up();
      await pointer2.up();
      await tester.pump();
    });
  });
}
```

**Explanation:**

- **`tester.tap()`**: Simulates a single tap. Finds the center of the widget and dispatches a down and up event.
- **`tester.longPress()`**: Holds down for 500ms (default long press duration) then releases.
- **`tester.drag()`**: Simulates a drag gesture. Moves from widget center by the specified offset. Good for testing drag handles or sliders.
- **`tester.scroll()`**: Specialized for scrollable widgets. Uses `Scrollable` semantics to scroll by offset.
- **`tester.fling()`**: Simulates a swipe with velocity. Important for testing scroll physics and page transitions.
- **`tester.pumpAndSettle()`**: Essential after animations. Runs `pump()` repeatedly until no more frames are scheduled (animations complete, timers settle).
- **Multi-touch**: Use `startGesture()` with different pointer IDs to simulate multiple simultaneous touches for pinch/spread gestures.

### **Text Input and Form Testing**

```dart
// File: test/presentation/widgets/form_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Form Interactions', () {
    testWidgets('text entry and validation', (WidgetTester tester) async {
      final formKey = GlobalKey<FormState>();
      String? submittedValue;
      
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Form(
              key: formKey,
              child: Column(
                children: [
                  TextFormField(
                    key: Key('email_field'),
                    decoration: InputDecoration(labelText: 'Email'),
                    validator: (value) {
                      if (value?.contains('@') != true) {
                        return 'Invalid email';
                      }
                      return null;
                    },
                    onSaved: (value) => submittedValue = value,
                  ),
                  ElevatedButton(
                    onPressed: () {
                      if (formKey.currentState?.validate() == true) {
                        formKey.currentState?.save();
                      }
                    },
                    child: Text('Submit'),
                  ),
                ],
              ),
            ),
          ),
        ),
      );
      
      // Enter text into field
      await tester.enterText(
        find.byKey(Key('email_field')),
        'test@example.com',
      );
      await tester.pump();
      
      // Verify text entered
      expect(find.text('test@example.com'), findsOneWidget);
      
      // Tap submit
      await tester.tap(find.text('Submit'));
      await tester.pump();
      
      // Verify no error message
      expect(find.text('Invalid email'), findsNothing);
      expect(submittedValue, equals('test@example.com'));
      
      // Clear and enter invalid email
      await tester.enterText(
        find.byKey(Key('email_field')),
        'invalid',
      );
      await tester.pump();
      
      // Submit again
      await tester.tap(find.text('Submit'));
      await tester.pump();
      
      // Verify error shown
      expect(find.text('Invalid email'), findsOneWidget);
    });
    
    testWidgets('checkbox and switch toggling', (WidgetTester tester) async {
      var isChecked = false;
      var isSwitched = false;
      
      await tester.pumpWidget(
        MaterialApp(
          home: StatefulBuilder(
            builder: (context, setState) {
              return Scaffold(
                body: Column(
                  children: [
                    Checkbox(
                      value: isChecked,
                      onChanged: (v) => setState(() => isChecked = v!),
                    ),
                    Switch(
                      value: isSwitched,
                      onChanged: (v) => setState(() => isSwitched = v),
                    ),
                  ],
                ),
              );
            },
          ),
        ),
      );
      
      // Tap checkbox (taps at center, which is the touch target)
      await tester.tap(find.byType(Checkbox));
      await tester.pump();
      expect(isChecked, isTrue);
      
      // Tap switch
      await tester.tap(find.byType(Switch));
      await tester.pump();
      expect(isSwitched, isTrue);
    });
    
    testWidgets('dropdown selection', (WidgetTester tester) async {
      String? selectedValue;
      
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: DropdownButtonFormField<String>(
              key: Key('dropdown'),
              items: ['A', 'B', 'C'].map((value) {
                return DropdownMenuItem(
                  value: value,
                  child: Text('Option $value'),
                );
              }).toList(),
              onChanged: (v) => selectedValue = v,
            ),
          ),
        ),
      );
      
      // Open dropdown
      await tester.tap(find.byKey(Key('dropdown')));
      await tester.pump(); // Open animation
      await tester.pump(const Duration(seconds: 1)); // Wait for menu
      
      // Tap an option
      await tester.tap(find.text('Option B'));
      await tester.pump();
      
      expect(selectedValue, equals('B'));
    });
    
    testWidgets('date picker interaction', (WidgetTester tester) async {
      DateTime? selectedDate;
      
      await tester.pumpWidget(
        MaterialApp(
          home: Builder(
            builder: (context) {
              return Scaffold(
                body: ElevatedButton(
                  onPressed: () async {
                    final date = await showDatePicker(
                      context: context,
                      initialDate: DateTime(2023, 1, 1),
                      firstDate: DateTime(2020),
                      lastDate: DateTime(2025),
                    );
                    selectedDate = date;
                  },
                  child: Text('Pick Date'),
                ),
              );
            },
          ),
        ),
      );
      
      // Open picker
      await tester.tap(find.text('Pick Date'));
      await tester.pump(); // Show dialog
      
      // Find and tap specific date in calendar
      // This depends on internal structure of DatePicker
      await tester.tap(find.text('15'));
      await tester.pump();
      
      // Tap OK button
      await tester.tap(find.text('OK'));
      await tester.pump();
      
      expect(selectedDate?.day, equals(15));
    });
  });
}
```

**Explanation:**

- **`tester.enterText()`**: Simulates typing into a `TextField` or `TextFormField`. Clears existing text first, then types new text. Triggers `onChanged` callbacks.
- **Form validation**: Test both valid and invalid inputs. Verify error messages appear and disappear correctly.
- **StatefulBuilder**: A utility widget for testing stateful interactions without creating separate StatefulWidget classes in tests.
- **Dropdowns**: Require two pumps after opening—one for the animation, one for the menu to render.
- **Date/Time pickers**: Use `showDatePicker` with `Builder` to get proper context. Finding specific dates requires knowing the internal structure of Material pickers.

---

## **29.4 Testing Different Screen Sizes and Orientations**

Responsive apps must work across device sizes. Widget tests can simulate tablets, phones, and orientation changes.

### **Responsive Layout Testing**

```dart
// File: test/presentation/responsive_layout_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Responsive Layouts', () {
    // Helper to set specific screen size
    Future<void> setScreenSize(
      WidgetTester tester,
      Size size, {
      double pixelDensity = 1.0,
    }) async {
      tester.binding.window.physicalSizeTestValue = size;
      tester.binding.window.devicePixelRatioTestValue = pixelDensity;
      // Trigger rebuild with new constraints
      await tester.pump();
    }
    
    testWidgets('shows navigation rail on tablet', (WidgetTester tester) async {
      // Tablet size (landscape)
      await setScreenSize(tester, Size(1200, 800));
      
      await tester.pumpWidget(
        MaterialApp(
          home: ResponsiveScaffold(),
        ),
      );
      
      // Should show NavigationRail on large screens
      expect(find.byType(NavigationRail), findsOneWidget);
      expect(find.byType(BottomNavigationBar), findsNothing);
    });
    
    testWidgets('shows bottom nav on phone', (WidgetTester tester) async {
      // Phone size (portrait)
      await setScreenSize(tester, Size(400, 800), pixelDensity: 2.0);
      
      await tester.pumpWidget(
        MaterialApp(
          home: ResponsiveScaffold(),
        ),
      );
      
      // Should show BottomNavigationBar on small screens
      expect(find.byType(BottomNavigationBar), findsOneWidget);
      expect(find.byType(NavigationRail), findsNothing);
    });
    
    testWidgets('adapts to orientation change', (WidgetTester tester) async {
      // Start in portrait
      await setScreenSize(tester, Size(400, 800));
      
      await tester.pumpWidget(
        MaterialApp(
        home: OrientationBuilder(
            builder: (context, orientation) {
              return Text('Orientation: $orientation');
            },
          ),
        ),
      );
      
      expect(find.text('Orientation: Orientation.portrait'), findsOneWidget);
      
      // Change to landscape
      await setScreenSize(tester, Size(800, 400));
      
      expect(find.text('Orientation: Orientation.landscape'), findsOneWidget);
    });
    
    testWidgets('adapts column count based on width', (WidgetTester tester) async {
      // Narrow screen - 1 column
      await setScreenSize(tester, Size(300, 600));
      await tester.pumpWidget(MaterialApp(home: GridLayout()));
      expect(find.byType(GridView), findsOneWidget);
      
      // Check crossAxisCount indirectly by finding items per row
      // This requires checking layout properties or specific responsive behavior
      
      // Wide screen - 3 columns
      await setScreenSize(tester, Size(900, 600));
      await tester.pumpWidget(MaterialApp(home: GridLayout()));
      
      // Verify different layout behavior
    });
    
    tearDown(() {
      // Reset window properties after each test
      // Note: addTearDown in individual tests is safer, 
      // but this shows group-level cleanup
    });
  });
}

class ResponsiveScaffold extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    final width = MediaQuery.of(context).size.width;
    final isTablet = width > 600;
    
    return Scaffold(
      body: Row(
        children: [
          if (isTablet)
            NavigationRail(
              destinations: [
                NavigationRailDestination(
                  icon: Icon(Icons.home),
                  label: Text('Home'),
                ),
              ],
              selectedIndex: 0,
            ),
          Expanded(child: Text('Content')),
        ],
      ),
      bottomNavigationBar: !isTablet
          ? BottomNavigationBar(
              items: [
                BottomNavigationBarItem(
                  icon: Icon(Icons.home),
                  label: 'Home',
                ),
              ],
            )
          : null,
    );
  }
}

class GridLayout extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    final width = MediaQuery.of(context).size.width;
    final columns = width > 600 ? 3 : 1;
    
    return GridView.count(
      crossAxisCount: columns,
      children: List.generate(6, (i) => Text('Item $i')),
    );
  }
}
```

**Explanation:**

- **Screen size override**: `physicalSizeTestValue` sets the screen dimensions. This affects `MediaQuery`, `LayoutBuilder`, and responsive breakpoints.
- **Pixel density**: `devicePixelRatioTestValue` affects how Flutter interprets physical pixels vs. logical pixels. Important for testing image resolution or text scaling.
- **Orientation simulation**: Swapping width and height values simulates rotation. `OrientationBuilder` responds to these changes.
- **Cleanup**: Always reset window properties in `tearDown` or `addTearDown` to prevent one test's screen size from affecting others.
- **Responsive breakpoints**: Test at common breakpoints (360px phone, 600px large phone, 840px tablet) to ensure layout adapts correctly.

---

## **29.5 Golden Tests (Visual Regression Testing)**

Golden tests capture screenshots of widgets and compare them against reference images ("goldens") to detect visual changes.

### **Creating and Maintaining Goldens**

```dart
// File: test/presentation/goldens/golden_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Golden Tests', () {
    testWidgets('Counter widget matches golden', (WidgetTester tester) async {
      // Build widget with specific theme
      await tester.pumpWidget(
        MaterialApp(
          theme: ThemeData.light(),
          home: Scaffold(
            body: Center(
              child: CounterWidget(),
            ),
          ),
        ),
      );
      
      // matchesGoldenFile compares against stored image
      await expectLater(
        find.byType(CounterWidget),
        matchesGoldenFile('counter_widget.png'),
      );
    });
    
    testWidgets('card component renders correctly', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Center(
              child: SizedBox(
                width: 300,
                height: 200,
                child: Card(
                  child: Padding(
                    padding: EdgeInsets.all(16),
                    child: Column(
                      children: [
                        Text('Title', style: TextStyle(fontSize: 24)),
                        Text('Subtitle'),
                      ],
                    ),
                  ),
                ),
              ),
            ),
          ),
        ),
      );
      
      // Wait for all shadows/elevation to render
      await tester.pumpAndSettle();
      
      await expectLater(
        find.byType(Card),
        matchesGoldenFile('card_component.png'),
      );
    });
    
    testWidgets('dark theme matches golden', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          theme: ThemeData.dark(),
          home: MyHomePage(),
        ),
      );
      
      await expectLater(
        find.byType(MyHomePage),
        matchesGoldenFile('dark_theme.png'),
      );
    });
    
    testWidgets('different screen sizes', (WidgetTester tester) async {
      // Test at specific size to ensure consistent screenshots
      tester.binding.window.physicalSizeTestValue = Size(400, 800);
      tester.binding.window.devicePixelRatioTestValue = 1.0;
      
      await tester.pumpWidget(
        MaterialApp(home: MyApp()),
      );
      
      await expectLater(
        find.byType(MyApp),
        matchesGoldenFile('mobile_portrait.png'),
      );
    });
  });
}

// Commands to run golden tests:

// Update goldens (create reference images):
// flutter test --update-goldens

// Run tests (compare against existing goldens):
// flutter test

// Run specific golden file:
// flutter test test/presentation/goldens/golden_test.dart
```

**Explanation:**

- **`matchesGoldenFile()`**: Compares the rendered widget against a PNG file stored in the test directory. If pixels differ, the test fails.
- **First run**: Use `--update-goldens` flag to generate initial reference images. These become your "gold standard."
- **Subsequent runs**: Tests compare current rendering against the golden file. Any difference (font, color, layout) causes failure.
- **Stability**: Always use `pumpAndSettle()` to ensure animations complete. Set specific screen sizes to avoid differences across devices.
- **Version control**: Commit golden files to Git. When intentional visual changes occur, update goldens with `--update-goldens` and commit new images.

---

## **29.6 Testing Animations and Transitions**

Animations require special handling to test intermediate states and completion.

### **Animation Testing Strategies**

```dart
// File: test/presentation/widgets/animation_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Animation Testing', () {
    testWidgets('fade animation completes', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: FadeTransitionWidget(),
        ),
      );
      
      // Initial state - widget might be invisible
      expect(find.text('Hello'), findsOneWidget);
      
      // Pump frames to advance animation
      // Each pump advances time by default (16ms per frame)
      await tester.pump(); // Start animation
      await tester.pump(Duration(milliseconds: 100)); // Partway
      await tester.pump(Duration(milliseconds: 400)); // Near end
      
      // Or use pumpAndSettle to wait for completion
      await tester.pumpAndSettle();
      
      // Verify final state
      expect(find.text('Hello'), findsOneWidget);
    });
    
    testWidgets('page transition animation', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: PageOne(),
        ),
      );
      
      // Navigate to page 2
      await tester.tap(find.text('Go to Page 2'));
      await tester.pump(); // Start transition
      
      // During transition, both pages might exist
      expect(find.text('Page 1'), findsOneWidget);
      expect(find.text('Page 2'), findsOneWidget);
      
      // Wait for transition to complete
      await tester.pumpAndSettle();
      
      // Now only page 2 should be visible
      expect(find.text('Page 1'), findsNothing);
      expect(find.text('Page 2'), findsOneWidget);
    });
    
    testWidgets('controlled animation timing', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: AnimatedContainer(
            duration: Duration(seconds: 1),
            color: Colors.red,
            child: Text('Animated'),
          ),
        ),
      );
      
      // Check at specific time intervals
      await tester.pump(Duration(milliseconds: 0));
      // Check initial state
      
      await tester.pump(Duration(milliseconds: 500));
      // Check halfway state
      
      await tester.pump(Duration(milliseconds: 1000));
      // Check completion
    });
    
    testWidgets('hero animation between routes', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: HeroSourcePage(),
        ),
      );
      
      // Find hero widget
      expect(find.byType(Hero), findsOneWidget);
      
      // Trigger navigation
      await tester.tap(find.byType(InkWell));
      await tester.pump();
      
      // Hero animation is running
      await tester.pump(Duration(milliseconds: 100));
      
      // Complete animation
      await tester.pumpAndSettle();
      
      // Verify on destination page
      expect(find.byType(HeroDestinationPage), findsOneWidget);
    });
  });
}

class FadeTransitionWidget extends StatefulWidget {
  @override
  _FadeTransitionWidgetState createState() => _FadeTransitionWidgetState();
}

class _FadeTransitionWidgetState extends State<FadeTransitionWidget>
    with SingleTickerProviderStateMixin {
  late AnimationController _controller;
  
  @override
  void initState() {
    super.initState();
    _controller = AnimationController(
      duration: Duration(seconds: 1),
      vsync: this,
    );
    _controller.forward();
  }
  
  @override
  Widget build(BuildContext context) {
    return FadeTransition(
      opacity: _controller,
      child: Text('Hello'),
    );
  }
  
  @override
  void dispose() {
    _controller.dispose();
    super.dispose();
  }
}
```

**Explanation:**

- **`pump()` with duration**: Advances the animation clock by the specified duration. Use this to test specific frames of an animation.
- **`pumpAndSettle()`**: Keeps pumping until no more frames are scheduled (animation completes, timers finish). Essential for most animation tests.
- **Transition testing**: During page transitions, both the outgoing and incoming widgets exist in the tree simultaneously. Use `pump()` to capture intermediate states.
- **Hero animations**: Test that shared elements animate between routes by checking existence at various time points.

---

## **29.7 Testing Complex Widget Trees and Custom Painters**

Complex UIs and custom rendering require specialized testing approaches.

### **Deep Widget Trees and Custom Painters**

```dart
// File: test/presentation/complex_widgets_test.dart
import 'package:flutter/material.dart';
import 'package:flutter_test/flutter_test.dart';

void main() {
  group('Complex Widgets', () {
    testWidgets('deeply nested widget finding', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: Scaffold(
            body: Container(
              child: Center(
                child: Column(
                  children: [
                    Row(
                      children: [
                        Padding(
                          padding: EdgeInsets.all(8),
                          child: Text('Deep Text'),
                        ),
                      ],
                    ),
                  ],
                ),
              ),
            ),
          ),
        ),
      );
      
      // Can find deeply nested text directly
      expect(find.text('Deep Text'), findsOneWidget);
      
      // Or find specific path
      expect(
        find.descendant(
          of: find.byType(Padding),
          matching: find.text('Deep Text'),
        ),
        findsOneWidget,
      );
    });
    
    testWidgets('custom painter verification', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: CustomPaint(
            size: Size(100, 100),
            painter: MyCustomPainter(),
          ),
        ),
      );
      
      // Custom painters render to canvas
      // We verify they exist and have correct size
      final customPaint = tester.widget<CustomPaint>(find.byType(CustomPaint));
      expect(customPaint.size, equals(Size(100, 100)));
      expect(customPaint.painter, isA<MyCustomPainter>());
    });
    
    testWidgets('listview with finders', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: ListView.builder(
            itemCount: 100,
            itemBuilder: (context, index) => ListTile(
              title: Text('Item $index'),
            ),
          ),
        ),
      );
      
      // Scroll to specific item not yet rendered
      await tester.scrollUntilVisible(
        find.text('Item 50'),
        500.0, // Scroll delta
      );
      
      // Now item 50 is visible
      expect(find.text('Item 50'), findsOneWidget);
      
      // Verify item 0 is scrolled off screen
      expect(find.text('Item 0'), findsNothing);
    });
    
    testWidgets('slivers and custom scroll', (WidgetTester tester) async {
      await tester.pumpWidget(
        MaterialApp(
          home: CustomScrollView(
            slivers: [
              SliverAppBar(
                title: Text('Sliver Title'),
              ),
              SliverList(
                delegate: SliverChildListDelegate([
                  Text('Item 1'),
                  Text('Item 2'),
                ]),
              ),
            ],
          ),
        ),
      );
      
      // Slivers are in the tree
      expect(find.byType(SliverAppBar), findsOneWidget);
      expect(find.text('Sliver Title'), findsOneWidget);
      expect(find.text('Item 1'), findsOneWidget);
    });
  });
}

class MyCustomPainter extends CustomPainter {
  @override
  void paint(Canvas canvas, Size size) {
    final paint = Paint()..color = Colors.blue;
    canvas.drawRect(Offset.zero & size, paint);
  }
  
  @override
  bool shouldRepaint(covariant CustomPainter oldDelegate) => false;
}
```

**Explanation:**

- **Deep nesting**: `find.text()` and other finders search the entire tree, so depth doesn't matter for finding. Use `descendant`/`ancestor` to verify specific relationships.
- **Custom painters**: Verify by checking the `CustomPaint` widget properties rather than the painted output (which requires golden tests).
- **`scrollUntilVisible()`**: Essential for long lists. Automatically scrolls until the finder locates the widget, handling the scroll physics correctly.
- **Slivers**: Test like regular widgets but be aware they're inside `Viewport` and `Scrollable` widgets.

---

## **Chapter Summary**

In this chapter, we covered comprehensive widget testing strategies:

### **Key Takeaways:**

1. **Widget Test Structure**: Use `testWidgets` with `WidgetTester`. Always wrap in `MaterialApp`/`CupertinoApp`. Call `pump()` after state changes and `pumpAndSettle()` after animations.

2. **Finders**: Use `find.text()` for content, `find.byType()` for widget classes, `find.byKey()` for stable identification (preferred), and `find.byIcon()` for icons. Chain with `descendant`/`ancestor` for tree navigation.

3. **Interactions**: `tap()` for buttons, `enterText()` for inputs, `drag()`/`fling()` for gestures, `scroll()` for scrollables. Use `longPress()` for context menus.

4. **Responsive Testing**: Override `physicalSizeTestValue` and `devicePixelRatioTestValue` to simulate phones, tablets, and orientations. Always reset in `tearDown`.

5. **Golden Tests**: Use `matchesGoldenFile()` with `--update-goldens` flag to create reference images. Essential for catching unintended visual changes.

6. **Animations**: Use `pump(duration)` to advance to specific frames, `pumpAndSettle()` to wait for completion. Test intermediate states for complex transitions.

7. **Complex Trees**: Use `scrollUntilVisible()` for long lists. Custom painters can be verified via widget properties or golden files.

### **Next Steps:**

Chapter 30 will cover **Integration & E2E Testing**, including:
- Setting up integration_test package
- Testing complete user flows across multiple screens
- Working with real devices and emulators
- Performance testing and profiling
- CI/CD integration for automated testing

---

**End of Chapter 29**

---

# **Next Chapter: Chapter 30 - Integration & E2E Testing**

Chapter 30 will focus on end-to-end testing strategies, complete user flow verification, device-specific testing, and integrating tests into CI/CD pipelines.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='28. unit_testing.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='30. integration_and_e2e_testing.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
